In [14]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
import os

In [15]:
def xor_bytes(a, b):
    """Return the XOR combination of two byte strings."""
    return bytes(x ^ y for x, y in zip(a, b))

In [36]:
def aes_128_ofb_encrypt(plaintext, key, iv):
    """
    Encrypt the plaintext using AES-128-OFB mode.

    Args:
        plaintext (bytes): The plaintext to be encrypted.
        key (bytes): The 16-byte (128-bit) encryption key.
        iv (bytes): The 16-byte (128-bit) initialization vector.

    Returns:
        bytes: The encrypted ciphertext.
    """
    # Check if the key and IV are 16 bytes
    if len(key) != 16:
        raise ValueError("Key must be 16 bytes.")
    if len(iv) != 16:
        raise ValueError("IV must be 16 bytes.")
    
    # Check if the plaintext is in bytes
    if type(plaintext) != bytes:
        raise ValueError("Plaintext must be bytes.")

    # Create the AES-128-ECB cipher
    aes_128_ECB = AES.new(key, AES.MODE_ECB)

    # Initialize the OFB state
    state = iv
    
    # Encrypt the plaintext
    ciphertext = b""
    for i in range(0, len(plaintext), 16):
        # Encrypt the state
        state = aes_128_ECB.encrypt(state)

        block = plaintext[i:i+16]

        # padd the block if it is less than 16 bytes
        #if len(block) < 16:
        #    block += b"\x00" * (16 - len(block))

        # XOR the state with the plaintext
        ciphertext += xor_bytes(state, block)

    return ciphertext


In [37]:
def aes_128_ofb_decrypt(ciphertext, key, iv):
    """
    Decrypt the ciphertext using AES-128-OFB mode.

    Args:
        ciphertext (bytes): The ciphertext to be decrypted.
        key (bytes): The 16-byte (128-bit) encryption key.
        iv (bytes): The 16-byte (128-bit) initialization vector.

    Returns:
        bytes: The decrypted plaintext.
    """
    # same as encrypt, just call it with the ciphertext
    plaintext = aes_128_ofb_encrypt(ciphertext, key, iv)
    
    # remove padding
    #plaintext = plaintext_padded.rstrip(b"\x00")

    return plaintext

In [39]:
# Test the AES-128-OFB encryption and decryption
key = get_random_bytes(16)  # Random 128-bit key
iv = get_random_bytes(16)   # Random 128-bit IV

# Plaintext must be at least 512 bits
plaintext_first = b'This is a 512-bit plaintext for testing AES-128-OFB encryption and decryption.'

# Encrypt
ciphertext_first = aes_128_ofb_encrypt(plaintext_first, key, iv)
print("Ciphertext:", ciphertext_first.hex())

# Decrypt
decrypted = aes_128_ofb_decrypt(ciphertext_first, key, iv)
print("Decrypted:", decrypted.decode())

Ciphertext: 31bf82a4bdba44324c45ade79fa110cf314277116e319766502c74f33fc6fc16a2d06eb601d2e28d3d989941d3465edbc8609afab42602799678f6a6d6298b330b7d58a1f711f5d09c9eb6c6be34
Decrypted: This is a 512-bit plaintext for testing AES-128-OFB encryption and decryption.


In [47]:
key = get_random_bytes(16)  # Random 128-bit key
iv = get_random_bytes(16)   # Random 128-bit IV

# Attack on user-chosen nonce N
plaintext_first = b'Attack on user-chosen nonce N.'
ciphertext_first = aes_128_ofb_encrypt(plaintext_first, key, iv)
print("First Ciphertext:", ciphertext_first.hex())

# Second encryption with the same IV
plaintext_second = b'..............................'
ciphertext_second = aes_128_ofb_encrypt(plaintext_second, key, iv)
print("Second Ciphertext:", ciphertext_second.hex())

# Attack
xor_messages = xor_bytes(ciphertext_first, ciphertext_second)
decrypted = xor_bytes(xor_messages, plaintext_second)
print("Decrypted:", decrypted.decode())

First Ciphertext: ab212c1e4f78f0e9c7a3bb54687240ce9b0286c481b9c179a9f1ec40cdce
Second Ciphertext: c47b7651023dfea887ade009232e4383dd43db8fc1b78138e9bca74eadce
Decrypted: Attack on user-chosen nonce N.
